In [1]:
import pandas as pd

In [4]:
import os
import pandas as pd

def generate_data_csv(csv_paths, output_path='data.csv'):
    """
    Create a unified CSV from multiple snr_results.csv files with specific metadata fields.

    Args:
        csv_paths (list): List of paths to snr_result CSV files.
        output_path (str): Output path for the merged data.csv file.
    """
    all_data = []

    for path in csv_paths:
        try:
            df = pd.read_csv(path)

            for _, row in df.iterrows():
                filename = row['filename']
                snr = row['snr_db']

                bonafide_status = 'Spoof' if 'mar_' in filename else 'Bonafide'

                entry = {
                    'BONAFIDE AUDIO FILE': filename,
                    'PLAYBACK DEVICE': '',
                    'RECORDING DEVICE': '',
                    'DISTANCE': '',
                    'ANGULAR ORIENTATION': '',
                    'ROOM ENVIRONMENT': '',
                    'GENDER': '',
                    'SNR': snr
                }

                all_data.append(entry)

        except Exception as e:
            print(f"Failed to process {path}: {e}")

    final_df = pd.DataFrame(all_data)
    final_df.to_csv(output_path, index=False)
    print(f"✅ Merged CSV saved to: {output_path}")

# 🔁 Example Usage
# generate_data_csv([
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/2M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/1M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/5M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/50CM snr_results.csv'
# ])

In [5]:
generate_data_csv([
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/2M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/1M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/5M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/50CM snr_results.csv'
])

✅ Merged CSV saved to: data.csv


In [1]:
! pip install transformers torchaudio librosa pesq soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 2.2 MB/s eta 0:00:0000:0100:01
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.9/285.9 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━

In [3]:
! pip install pandas 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 1.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import librosa
import pandas as pd
from pesq import pesq
from transformers import pipeline
import soundfile as sf

# Load Whisper model
whisper_pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def transcribe(audio_path):
    try:
        return whisper_pipe(audio_path)['text']
    except Exception as e:
        print(f"❌ Whisper failed on {audio_path}: {e}")
        return "ERROR"

def compute_pesq(ref_path, deg_path, sample_rate=16000):
    try:
        ref, sr1 = librosa.load(ref_path, sr=sample_rate)
        deg, sr2 = librosa.load(deg_path, sr=sample_rate)

        min_len = min(len(ref), len(deg))
        ref, deg = ref[:min_len], deg[:min_len]

        pesq_val = pesq(sample_rate, ref, deg, 'wb')  # 'wb' = wideband
        return pesq_val
    except Exception as e:
        print(f"❌ Error comparing {ref_path} and {deg_path}: {e}")
        return None

def process_folder_by_sequence(ref_folder, deg_folder, output_csv="pesq_results.csv"):
    ref_files = sorted([f for f in os.listdir(ref_folder) if f.endswith('.wav')])
    deg_files = sorted([f for f in os.listdir(deg_folder) if not f.startswith('.')])  # exclude hidden files

    results = []

    for i in range(min(len(ref_files), len(deg_files))):
        ref_path = os.path.join(ref_folder, ref_files[i])
        deg_path = os.path.join(deg_folder, deg_files[i])

        print(f"🔍 Processing:\nRef: {ref_files[i]}\nDeg: {deg_files[i]}")

        pesq_score = compute_pesq(ref_path, deg_path)
        ref_text = transcribe(ref_path)
        deg_text = transcribe(deg_path)

        results.append({
            "Ref File": ref_files[i],
            "Deg File": deg_files[i],
            "PESQ": pesq_score,
            "Ref Transcript": ref_text,
            "Deg Transcript": deg_text
        })

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"✅ PESQ results saved to {output_csv}")


Device set to use mps:0


In [8]:
process_folder("/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/real", "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/spoofed")
process_folder_by_sequence(
    "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/real",
    "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/spoofed",
    output_csv="pesq_results.csv"
)

Missing degraded file for 10.wav
Missing degraded file for 9.wav
Missing degraded file for 8.wav
Missing degraded file for 6.wav
Missing degraded file for 7.wav
Missing degraded file for 5.wav
Missing degraded file for 4.wav
Missing degraded file for 1.wav
Missing degraded file for 3.wav
Missing degraded file for 2.wav
✅ PESQ results saved to pesq_results.csv
🔍 Processing:
Ref: 1.wav
Deg: 1.m4a


/var/folders/mz/_mgl45y53018fg9xtlkwt0_m0000gn/T/ipykernel_69099/3699764797.py:21: UserWarning: PySoundFile failed. Trying audioread instead.
  deg, sr2 = librosa.load(deg_path, sr=sample_rate)
/Users/rohitlabade/.pyenv/versions/3.9.18/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


KeyboardInterrupt: 